In [1]:
pip install -U "tensorflow-text==2.13.*"

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
df=pd.read_csv('/content/Sora_LREC2020_biasedsentences.csv')
df.shape

(215, 56)

In [4]:
df.head(5)

,id_event,event,date_event,id_article,source,source_bias,url,ref,ref_url,article_bias,...,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19
0,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,1,...,"[10]: Johnson's wife, Rebecca, announced Thurs...",[11]: She spent the day at a funeral home arra...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc..."
1,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,"[10]: Johnson's wife, Rebecca, announced Thurs...",[11]: She spent the day at a funeral home arra...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc..."
2,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,"[10]: Johnson's wife, Rebecca, announced Thurs...",[11]: She spent the day at a funeral home arra...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc..."
3,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,"[10]: Johnson's wife, Rebecca, announced Thurs...",[11]: She spent the day at a funeral home arra...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc..."
4,1,Johnson,2017-12-15,2.0,002_CNN_1,left,http://www.cnn.com/2017/12/14/us/kentucky-stat...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,4,...,"[10]: The accuser, identified as Maranda Richm...",[11]: She said she was staying in a living are...,[12]: The center got its hands on police docum...,"[13]: Johnson denied the allegations, a day af...","[14]: ""This allegation concerning this young g...","[15]: ""As a matter of fact, some of this I hea...",[16]: The same day Johnson held a news confere...,[17]: Johnson posted a message on his Facebook...,[18]: The post appears to have been deleted.,"[19]: ""GOD and only GOD knows the truth, nothi..."


In [5]:
# check each event's data size
print(df['id_event'].unique())
print(df['id_event'].value_counts())

[1 2 3 4]
id_event
3    58
2    57
1    50
4    50
Name: count, dtype: int64


In [6]:
# check preknow=yes and no's data size
print(df['preknow'].unique())
print(df['preknow'].value_counts())

['yes' 'no']
preknow
no     156
yes     59
Name: count, dtype: int64


In [7]:
print(df.groupby(['preknow', 'id_event']).size())

preknow  id_event
no       1           40
         2           36
         3           42
         4           38
yes      1           10
         2           21
         3           16
         4           12
dtype: int64


In [8]:
# Calculate all 'preknow=no' and 'preknow=yes' ratings' mean
preknow_no_mean = (
    df[df['preknow'] == 'no']
    .groupby('id_article')['article_bias']
    .mean()
    .rename("preknow_no_mean")
)

preknow_yes_mean = (
    df[df['preknow'] == 'yes']
    .groupby('id_article')['article_bias']
    .mean()
    .rename("preknow_yes_mean")
)

preknow_no_majority = (
    df[df['preknow'] == 'no']
    .groupby('id_article')['article_bias']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    .rename("preknow_no_majority")
)

preknow_yes_majority = (
    df[df['preknow'] == 'yes']
    .groupby('id_article')['article_bias']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    .rename("preknow_yes_majority")
)

df = df.merge(preknow_no_mean, on='id_article', how='left')
df = df.merge(preknow_yes_mean, on='id_article', how='left')
df = df.merge(preknow_no_majority, on='id_article', how='left')
df = df.merge(preknow_yes_majority, on='id_article', how='left')

In [9]:
df['prio_aggregation'] =  df['preknow_yes_majority'].fillna(df['preknow_no_majority'])
print(df['prio_aggregation'].value_counts())

prio_aggregation
2.0    92
1.0    80
3.0    43
Name: count, dtype: int64


In [10]:
df['preknow_yes_majority'] = df['preknow_yes_majority'].fillna(0)
df['preknow_no_majority'] = df['preknow_no_majority'].fillna(0)

In [11]:
weight_yes = 0.8
weight_no = 0.2

df['weighted_combination'] = (
    weight_yes * df['preknow_yes_majority'] +
    weight_no * df['preknow_no_majority']
)

In [12]:
df.head(10)

,id_event,event,date_event,id_article,source,source_bias,url,ref,ref_url,article_bias,...,s16,s17,s18,s19,preknow_no_mean,preknow_yes_mean,preknow_no_majority,preknow_yes_majority,prio_aggregation,weighted_combination
0,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,1,...,[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",3.0,1.0,3,1.0,1.0,1.4
1,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",3.0,1.0,3,1.0,1.0,1.4
2,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",3.0,1.0,3,1.0,1.0,1.4
3,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",3.0,1.0,3,1.0,1.0,1.4
4,1,Johnson,2017-12-15,2.0,002_CNN_1,left,http://www.cnn.com/2017/12/14/us/kentucky-stat...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,4,...,[16]: The same day Johnson held a news confere...,[17]: Johnson posted a message on his Facebook...,[18]: The post appears to have been deleted.,"[19]: ""GOD and only GOD knows the truth, nothi...",3.0,NaN,3,0.0,3.0,0.6
5,1,Johnson,2017-12-15,2.0,002_CNN_1,left,http://www.cnn.com/2017/12/14/us/kentucky-stat...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,[16]: The same day Johnson held a news confere...,[17]: Johnson posted a message on his Facebook...,[18]: The post appears to have been deleted.,"[19]: ""GOD and only GOD knows the truth, nothi...",3.0,NaN,3,0.0,3.0,0.6
6,1,Johnson,2017-12-15,2.0,002_CNN_1,left,http://www.cnn.com/2017/12/14/us/kentucky-stat...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,2,...,[16]: The same day Johnson held a news confere...,[17]: Johnson posted a message on his Facebook...,[18]: The post appears to have been deleted.,"[19]: ""GOD and only GOD knows the truth, nothi...",3.0,NaN,3,0.0,3.0,0.6
7,1,Johnson,2017-12-15,2.0,002_CNN_1,left,http://www.cnn.com/2017/12/14/us/kentucky-stat...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,[16]: The same day Johnson held a news confere...,[17]: Johnson posted a message on his Facebook...,[18]: The post appears to have been deleted.,"[19]: ""GOD and only GOD knows the truth, nothi...",3.0,NaN,3,0.0,3.0,0.6
8,1,Johnson,2017-12-15,3.0,003_NBCNews,left-center,https://www.nbcnews.com/storyline/sexual-misco...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,1,...,"[16]: Before his suicide, Johnson made nationa...",[17]: That didn't stop him from getting electe...,"[18]: Johnson claimed to have a ""doctorate of ...",[19]: The Kentucky Center for Investigative Re...,3.0,1.0,3,1.0,1.0,1.4
9,1,Johnson,2017-12-15,3.0,003_NBCNews,left-center,https://www.nbcnews.com/storyline/sexual-misco...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,"[16]: Before his suicide, Johnson made nationa...",[17]: That didn't stop him from getting electe...,"[18]: Johnson claimed to have a ""doctorate of ...",[19]: The Kentucky Center for Investigative Re...,3.0,1.0,3,1.0,1.0,1.4


In [13]:
# Concate s0 to s19 into a single column
sentence_columns = [f's{i}' for i in range(20)]  # List of sentence columns
df['full_article'] = df[sentence_columns].fillna('').agg(' '.join, axis=1)

In [14]:
df['weighted_combination'] = df['weighted_combination'].round()

In [15]:
prio_df = df[['id_article','doctitle','full_article','id_event','prio_aggregation']].drop_duplicates()
prio_df.head(10)

,id_article,doctitle,full_article,id_event,prio_aggregation
0,1.0,Dan Johnson suicide: Lawmaker accused of moles...,[0]: LOUISVILLE - Dan Johnson posted a final m...,1,1.0
4,2.0,Kentucky lawmaker Dan Johnson killed himself a...,[0]: (CNN) A Kentucky lawmaker accused of sexu...,1,3.0
8,3.0,"Dan Johnson, Kentucky lawmaker who killed hims...",[0]: The Kentucky preacher-turned-politician w...,1,1.0
12,4.0,"Matt Bevin, Rand Paul Respond to Dan Johnson's...",[0]: After Kentucky State Rep. Dan Johnson die...,1,2.0
16,5.0,Widow of Kentucky State Rep. Dan Johnson says ...,[0]: The wife of Kentucky State Rep. Dan Johns...,1,3.0
20,6.0,Widow of Kentucky state rep Dan Johnson plans ...,"[0]: ""Dan is gone,"" says Rebecca Johnson, wife...",1,1.0
25,7.0,Wife to run for seat of Kentucky lawmaker who ...,[0]: The widow of the Kentucky lawmaker who ki...,1,1.0
30,8.0,"Kentucky rep, gun-toting preacher commits suic...","[0]: Kentucky state Rep. Dan Johnson, who had ...",1,1.0
35,9.0,Kentucky Lawmaker Kills Self After Sexual Assa...,[0]: A Kentucky state representative committed...,1,1.0
40,10.0,Top US politician Dan Johnson kills himself fo...,[0]: A TOP politician killed himself after lea...,1,1.0


In [16]:
# remove '[0]: '
prio_df['full_article'] = prio_df['full_article'].str.replace('[0]: ', '')

In [17]:
print(prio_df.groupby('prio_aggregation').size())

prio_aggregation
1.0    17
2.0    19
3.0    10
dtype: int64


In [18]:
# only extract unique id_article, preknow_agg, and concact s0-19
unique_df = df[['id_article','doctitle','full_article','id_event','weighted_combination']].drop_duplicates()

In [19]:
unique_df.head(10)

,id_article,doctitle,full_article,id_event,weighted_combination
0,1.0,Dan Johnson suicide: Lawmaker accused of moles...,[0]: LOUISVILLE - Dan Johnson posted a final m...,1,1.0
4,2.0,Kentucky lawmaker Dan Johnson killed himself a...,[0]: (CNN) A Kentucky lawmaker accused of sexu...,1,1.0
8,3.0,"Dan Johnson, Kentucky lawmaker who killed hims...",[0]: The Kentucky preacher-turned-politician w...,1,1.0
12,4.0,"Matt Bevin, Rand Paul Respond to Dan Johnson's...",[0]: After Kentucky State Rep. Dan Johnson die...,1,2.0
16,5.0,Widow of Kentucky State Rep. Dan Johnson says ...,[0]: The wife of Kentucky State Rep. Dan Johns...,1,3.0
20,6.0,Widow of Kentucky state rep Dan Johnson plans ...,"[0]: ""Dan is gone,"" says Rebecca Johnson, wife...",1,1.0
25,7.0,Wife to run for seat of Kentucky lawmaker who ...,[0]: The widow of the Kentucky lawmaker who ki...,1,1.0
30,8.0,"Kentucky rep, gun-toting preacher commits suic...","[0]: Kentucky state Rep. Dan Johnson, who had ...",1,1.0
35,9.0,Kentucky Lawmaker Kills Self After Sexual Assa...,[0]: A Kentucky state representative committed...,1,1.0
40,10.0,Top US politician Dan Johnson kills himself fo...,[0]: A TOP politician killed himself after lea...,1,1.0


In [20]:
# remove '[0]: '
unique_df['full_article'] = unique_df['full_article'].str.replace('[0]: ', '')

In [21]:
unique_df.head(10)

,id_article,doctitle,full_article,id_event,weighted_combination
0,1.0,Dan Johnson suicide: Lawmaker accused of moles...,LOUISVILLE - Dan Johnson posted a final messag...,1,1.0
4,2.0,Kentucky lawmaker Dan Johnson killed himself a...,(CNN) A Kentucky lawmaker accused of sexually ...,1,1.0
8,3.0,"Dan Johnson, Kentucky lawmaker who killed hims...",The Kentucky preacher-turned-politician who ki...,1,1.0
12,4.0,"Matt Bevin, Rand Paul Respond to Dan Johnson's...",After Kentucky State Rep. Dan Johnson died Wed...,1,2.0
16,5.0,Widow of Kentucky State Rep. Dan Johnson says ...,"The wife of Kentucky State Rep. Dan Johnson, w...",1,3.0
20,6.0,Widow of Kentucky state rep Dan Johnson plans ...,"""Dan is gone,"" says Rebecca Johnson, wife of d...",1,1.0
25,7.0,Wife to run for seat of Kentucky lawmaker who ...,The widow of the Kentucky lawmaker who killed ...,1,1.0
30,8.0,"Kentucky rep, gun-toting preacher commits suic...","Kentucky state Rep. Dan Johnson, who had built...",1,1.0
35,9.0,Kentucky Lawmaker Kills Self After Sexual Assa...,A Kentucky state representative committed suic...,1,1.0
40,10.0,Top US politician Dan Johnson kills himself fo...,A TOP politician killed himself after leaving ...,1,1.0


In [22]:
unique_df.shape

(46, 5)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
# Split into train test and val set
train_texts, tmp_texts, train_labels, tmp_labels = train_test_split(
    prio_df['full_article'], prio_df['prio_aggregation'], test_size=0.2, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    tmp_texts,tmp_labels, test_size=0.5, random_state=42
)

In [25]:
# Convert to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_texts.values, train_labels.values))
test_dataset = tf.data.Dataset.from_tensor_slices((test_texts.values, test_labels.values))
val_dataset = tf.data.Dataset.from_tensor_slices((val_texts.values, val_labels.values))


# Shuffle and batch the datasets
batch_size = 8 #10-20 for small dats size
train_dataset = train_dataset.shuffle(len(train_texts)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [26]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [27]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [28]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(4, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)


In [29]:
classifier_model = build_classifier_model()

In [30]:
# Display the model summary
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [89]:
from tensorflow.keras.callbacks import EarlyStopping

batch_size = 20
classifier_model = build_classifier_model()
# Compile the model
classifier_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with EarlyStopping
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    #callbacks=[early_stopping]
)


Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/20
5/5 [==============================] - 10s 410ms/step - loss: 1.7906 - accuracy: 0.0833 - val_loss: 2.2159 - val_accuracy: 0.4000
Epoch 2/20
5/5 [==============================] - 1s 294ms/step - loss: 1.1927 - accuracy: 0.4444 - val_loss: 2.2994 - val_accuracy: 0.0000e+00
Epoch 3/20
5/5 [==============================] - 1s 297ms/step - loss: 1.1063 - accuracy: 0.6111 - val_loss: 2.0098 - val_accuracy: 0.2000
Epoch 4/20
5/5 [==============================] - 2s 327ms/step - loss: 0.9452 - accuracy: 0.5000 - val_loss: 1.7577 - val_accuracy: 0.2000
Epoch 5/20
5/5 [==============================] - 2s 326ms/step - loss: 0.8588 - accuracy: 0.6667 - val_loss: 1.6621 - val_accuracy: 0.2000
Epoch 6/20
5/5 [==============================] - 2s 296ms/step - loss: 0.8164 - accuracy: 0.7222 - val_loss: 1.6819 - val_accuracy: 0.2000
Epoch 7/20
5/5 [==============================] - 2s 306ms/step

In [90]:
# Evaluate the model on the test dataset
loss, accuracy = classifier_model.evaluate(test_dataset)

# Print the test accuracy
print(f"Test Accuracy: {accuracy:.2f}")


1/1 [==============================] - 0s 82ms/step - loss: 1.0122 - accuracy: 0.6000
Test Accuracy: 0.60


# Using weights

In [23]:
# Split into train test and val set
train_texts, tmp_texts, train_labels, tmp_labels = train_test_split(
    unique_df['full_article'], unique_df['weighted_combination'], test_size=0.2, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    tmp_texts,tmp_labels, test_size=0.5, random_state=42
)

In [26]:
# Convert to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_texts.values, train_labels.values))
test_dataset = tf.data.Dataset.from_tensor_slices((test_texts.values, test_labels.values))
val_dataset = tf.data.Dataset.from_tensor_slices((val_texts.values, val_labels.values))


# Shuffle and batch the datasets
batch_size = 8 #10-20 for small dats size
train_dataset = train_dataset.shuffle(len(train_texts)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [34]:
from tensorflow.keras.callbacks import EarlyStopping

batch_size = 20
classifier_model = build_classifier_model()
# Compile the model
classifier_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with EarlyStopping
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stopping]
)


Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/20
22/22 [==============================] - 16s 364ms/step - loss: 1.3988 - accuracy: 0.3605 - val_loss: 1.3404 - val_accuracy: 0.4186
Epoch 2/20
22/22 [==============================] - 7s 335ms/step - loss: 1.2949 - accuracy: 0.4302 - val_loss: 1.1195 - val_accuracy: 0.4884
Epoch 3/20
22/22 [==============================] - 7s 337ms/step - loss: 1.2348 - accuracy: 0.3953 - val_loss: 1.1270 - val_accuracy: 0.5349
Epoch 4/20
22/22 [==============================] - 7s 329ms/step - loss: 1.1913 - accuracy: 0.4360 - val_loss: 1.0734 - val_accuracy: 0.6047
Epoch 5/20
22/22 [==============================] - 7s 339ms/step - loss: 1.2034 - accuracy: 0.4186 - val_loss: 1.1014 - val_accuracy: 0.5581
Epoch 6/20
22/22 [==============================] - 7s 324ms/step - loss: 1.2057 - accuracy: 0.4244 - val_loss: 1.1430 - val_accuracy: 0.4884
Epoch 7/20
22/22 [==============================] - 7s 

In [35]:
# Evaluate the model on the test dataset
loss, accuracy = classifier_model.evaluate(test_dataset)

# Print the test accuracy
print(f"Test Accuracy: {accuracy:.2f}")


6/6 [==============================] - 0s 79ms/step - loss: 1.0734 - accuracy: 0.6047
Test Accuracy: 0.60


# Treating all the annotations equally without preknown

In [36]:
df=pd.read_csv('/content/Sora_LREC2020_biasedsentences.csv')
df.shape

(215, 56)

In [37]:
labels = df['article_bias']

In [38]:
# Concate s0 to s19 into a single column
sentence_columns = [f's{i}' for i in range(20)]  # List of sentence columns
df['full_article'] = df[sentence_columns].fillna('').agg(' '.join, axis=1)

In [39]:
# remove '[0]: '
df['full_article'] = df['full_article'].str.replace('[0]: ', '')

In [40]:
df.head(5)

,id_event,event,date_event,id_article,source,source_bias,url,ref,ref_url,article_bias,...,s11,s12,s13,s14,s15,s16,s17,s18,s19,full_article
0,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,1,...,[11]: She spent the day at a funeral home arra...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",LOUISVILLE - Dan Johnson posted a final messag...
1,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,[11]: She spent the day at a funeral home arra...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",LOUISVILLE - Dan Johnson posted a final messag...
2,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,[11]: She spent the day at a funeral home arra...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",LOUISVILLE - Dan Johnson posted a final messag...
3,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,[11]: She spent the day at a funeral home arra...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",LOUISVILLE - Dan Johnson posted a final messag...
4,1,Johnson,2017-12-15,2.0,002_CNN_1,left,http://www.cnn.com/2017/12/14/us/kentucky-stat...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,4,...,[11]: She said she was staying in a living are...,[12]: The center got its hands on police docum...,"[13]: Johnson denied the allegations, a day af...","[14]: ""This allegation concerning this young g...","[15]: ""As a matter of fact, some of this I hea...",[16]: The same day Johnson held a news confere...,[17]: Johnson posted a message on his Facebook...,[18]: The post appears to have been deleted.,"[19]: ""GOD and only GOD knows the truth, nothi...",(CNN) A Kentucky lawmaker accused of sexually ...


In [41]:
unique_df = df[['id_article','doctitle','full_article','id_event','article_bias']]
unique_df['article_bias'] = unique_df['article_bias'] - 1

unique_df

<ipython-input-41-ffe9666045bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['article_bias'] = unique_df['article_bias'] - 1


,id_article,doctitle,full_article,id_event,article_bias
0,1.0,Dan Johnson suicide: Lawmaker accused of moles...,LOUISVILLE - Dan Johnson posted a final messag...,1,0
1,1.0,Dan Johnson suicide: Lawmaker accused of moles...,LOUISVILLE - Dan Johnson posted a final messag...,1,2
2,1.0,Dan Johnson suicide: Lawmaker accused of moles...,LOUISVILLE - Dan Johnson posted a final messag...,1,2
3,1.0,Dan Johnson suicide: Lawmaker accused of moles...,LOUISVILLE - Dan Johnson posted a final messag...,1,2
4,2.0,Kentucky lawmaker Dan Johnson killed himself a...,(CNN) A Kentucky lawmaker accused of sexually ...,1,3
...,...,...,...,...,...
210,46.0,North Korea and U.S. are meeting for talks,The United States had previously held that the...,4,1
211,46.0,North Korea and U.S. are meeting for talks,The United States had previously held that the...,4,0
212,46.0,North Korea and U.S. are meeting for talks,The United States had previously held that the...,4,1
213,46.0,North Korea and U.S. are meeting for talks,The United States had previously held that the...,4,1


In [42]:
# Split into train test and val set
train_texts, tmp_texts, train_labels, tmp_labels = train_test_split(
    unique_df['full_article'], unique_df['article_bias'], test_size=0.2, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    tmp_texts,tmp_labels, test_size=0.5, random_state=42
)

In [43]:
# Convert to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_texts.values, train_labels.values))
test_dataset = tf.data.Dataset.from_tensor_slices((test_texts.values, test_labels.values))
val_dataset = tf.data.Dataset.from_tensor_slices((val_texts.values, val_labels.values))


# Shuffle and batch the datasets
batch_size = 8 #10-20 for small dats size
train_dataset = train_dataset.shuffle(len(train_texts)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [44]:
batch_size = 20
classifier_model = build_classifier_model()
# Compile the model
classifier_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

# Train the model with EarlyStopping
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    #callbacks=[early_stopping]
)


Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/20
22/22 [==============================] - 15s 356ms/step - loss: 1.4268 - accuracy: 0.3837 - val_loss: 1.1020 - val_accuracy: 0.4651
Epoch 2/20
22/22 [==============================] - 7s 320ms/step - loss: 1.2420 - accuracy: 0.3953 - val_loss: 1.1097 - val_accuracy: 0.4419
Epoch 3/20
22/22 [==============================] - 7s 341ms/step - loss: 1.2383 - accuracy: 0.4186 - val_loss: 1.1120 - val_accuracy: 0.4419
Epoch 4/20
22/22 [==============================] - 7s 321ms/step - loss: 1.2357 - accuracy: 0.4360 - val_loss: 1.0564 - val_accuracy: 0.5814
Epoch 5/20
22/22 [==============================] - 7s 338ms/step - loss: 1.2169 - accuracy: 0.4535 - val_loss: 1.1271 - val_accuracy: 0.4651
Epoch 6/20
22/22 [==============================] - 7s 327ms/step - loss: 1.2119 - accuracy: 0.3895 - val_loss: 1.1171 - val_accuracy: 0.5814
Epoch 7/20
22/22 [==============================] - 7s 

In [45]:
# Evaluate the model on the test dataset
loss, accuracy = classifier_model.evaluate(test_dataset)

# Print the test accuracy
print(f"Test Accuracy: {accuracy:.2f}")


6/6 [==============================] - 1s 94ms/step - loss: 1.3483 - accuracy: 0.4186
Test Accuracy: 0.42


No included + extra data

In [59]:
df_extra = pd.read_excel("/content/final_labels_SG2.xlsx")
df_extra.shape

(3674, 8)

In [71]:
df_extra = df_extra[df_extra['label_opinion']!= 'No agreement']
df_extra['label_bias_0-2'] = df_extra['label_opinion'].map({"Expresses writer’s opinion":2,"Somewhat factual but also opinionated":1,'Entirely factual':0})
df_extra.head(5)

<ipython-input-71-e0994d5815ac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extra['label_bias_0-2'] = df_extra['label_opinion'].map({"Expresses writer’s opinion":2,"Somewhat factual but also opinionated":1,'Entirely factual':0})


,text,news_link,outlet,topic,type,label_bias,label_opinion,biased_words,label_bias_0-2
0,"""Orange Is the New Black"" star Yael Stone is r...",https://www.foxnews.com/entertainment/australi...,Fox News,environment,right,Non-biased,Entirely factual,[],0
1,"""We have one beautiful law,"" Trump recently sa...",https://www.alternet.org/2020/06/law-and-order...,Alternet,gun control,left,Biased,Somewhat factual but also opinionated,"['bizarre', 'characteristically']",1
2,"...immigrants as criminals and eugenics, all o...",https://www.nbcnews.com/news/latino/after-step...,MSNBC,white-nationalism,left,Biased,Expresses writer’s opinion,"['criminals', 'fringe', 'extreme']",2
3,...we sounded the alarm in the early months of...,https://www.alternet.org/2019/07/fox-news-has-...,Alternet,white-nationalism,left,Biased,Somewhat factual but also opinionated,[],1
4,[Black Lives Matter] is essentially a non-fals...,http://feedproxy.google.com/~r/breitbart/~3/-v...,Breitbart,marriage-equality,NaN,Biased,Expresses writer’s opinion,['cult'],2


In [61]:
df=pd.read_csv('/content/Sora_LREC2020_biasedsentences.csv')
df.shape

(215, 56)

In [74]:
#In this version we only have biased , no biased or slightly biased
df['label_bias_0-2'] = df['article_bias'].map({1:0,2:1,3:2,4:2})
df.head(5)

,id_event,event,date_event,id_article,source,source_bias,url,ref,ref_url,article_bias,...,s12,s13,s14,s15,s16,s17,s18,s19,label_bias_0-2,full_article
0,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,1,...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",0,[0]: LOUISVILLE - Dan Johnson posted a final m...
1,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",2,[0]: LOUISVILLE - Dan Johnson posted a final m...
2,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",2,[0]: LOUISVILLE - Dan Johnson posted a final m...
3,1,Johnson,2017-12-15,1.0,001_Washington Post_1,left-center,https://www.washingtonpost.com/news/post-natio...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,3,...,"[12]: ""Dan is gone but the story of his life i...","[13]: ""These high-tech lynchings based on lies...",[14]: I've been fighting behind my husband for...,"[15]: Johnson's death shook his family, friend...",[16]: Gov.,"[17]: Matt Bevin (R) wrote on Twitter: ""My hea...","[18]: [White House urges Roy Moore to concede,...","[19]: The tumult began Monday, when the Kentuc...",2,[0]: LOUISVILLE - Dan Johnson posted a final m...
4,1,Johnson,2017-12-15,2.0,002_CNN_1,left,http://www.cnn.com/2017/12/14/us/kentucky-stat...,Reuters,https://www.reuters.com/article/us-kentucky-jo...,4,...,[12]: The center got its hands on police docum...,"[13]: Johnson denied the allegations, a day af...","[14]: ""This allegation concerning this young g...","[15]: ""As a matter of fact, some of this I hea...",[16]: The same day Johnson held a news confere...,[17]: Johnson posted a message on his Facebook...,[18]: The post appears to have been deleted.,"[19]: ""GOD and only GOD knows the truth, nothi...",2,[0]: (CNN) A Kentucky lawmaker accused of sexu...


In [63]:
sentence_columns = [f's{i}' for i in range(20)]  # List of sentence columns
df['full_article'] = df[sentence_columns].fillna('').agg(' '.join, axis=1)
unique_df = df[['id_article','doctitle','full_article','id_event','label_bias_0-2']].drop_duplicates()
unique_df['full_article'] = unique_df['full_article'].str.replace('[0]: ', '')
unique_df.head(5)

,id_article,doctitle,full_article,id_event,label_bias_0-2
0,1.0,Dan Johnson suicide: Lawmaker accused of moles...,LOUISVILLE - Dan Johnson posted a final messag...,1,0
1,1.0,Dan Johnson suicide: Lawmaker accused of moles...,LOUISVILLE - Dan Johnson posted a final messag...,1,2
4,2.0,Kentucky lawmaker Dan Johnson killed himself a...,(CNN) A Kentucky lawmaker accused of sexually ...,1,2
6,2.0,Kentucky lawmaker Dan Johnson killed himself a...,(CNN) A Kentucky lawmaker accused of sexually ...,1,1
8,3.0,"Dan Johnson, Kentucky lawmaker who killed hims...",The Kentucky preacher-turned-politician who ki...,1,0


In [75]:
#use df_extra as training set and df as test set

train_dataset = tf.data.Dataset.from_tensor_slices((df_extra['text'].values, df_extra['label_bias_0-2'].values))
test_dataset = tf.data.Dataset.from_tensor_slices((unique_df['full_article'].values, unique_df['label_bias_0-2'].values))


# Shuffle and batch the datasets
batch_size = 8 #10-20 for small dats size
train_dataset = train_dataset.shuffle(len(train_texts)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [76]:
batch_size = 20
classifier_model = build_classifier_model()
# Compile the model
classifier_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

# Train the model with EarlyStopping
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(
    train_dataset,
    #validation_data=test_dataset,
    epochs=20,
    callbacks=[early_stopping]
)


Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/20
433/433 [==============================] - 142s 310ms/step - loss: 0.8408 - accuracy: 0.6135 - val_loss: 1.5798 - val_accuracy: 0.3036
Epoch 2/20
433/433 [==============================] - 134s 309ms/step - loss: 0.6239 - accuracy: 0.7264 - val_loss: 1.6606 - val_accuracy: 0.3304
Epoch 3/20
433/433 [==============================] - 134s 309ms/step - loss: 0.5333 - accuracy: 0.7747 - val_loss: 1.9315 - val_accuracy: 0.3214
Epoch 4/20
433/433 [==============================] - 136s 313ms/step - loss: 0.4272 - accuracy: 0.8261 - val_loss: 2.1884 - val_accuracy: 0.3214
Epoch 5/20
433/433 [==============================] - 135s 313ms/step - loss: 0.3341 - accuracy: 0.8698 - val_loss: 2.4575 - val_accuracy: 0.3125
Epoch 6/20
433/433 [==============================] - 134s 310ms/step - loss: 0.2396 - accuracy: 0.9069 - val_loss: 2.6556 - val_accuracy: 0.3036
Epoch 7/20
433/433 [============

In [77]:
# Evaluate the model on the test dataset
loss, accuracy = classifier_model.evaluate(test_dataset)

# Print the test accuracy
print(f"Test Accuracy: {accuracy:.2f}")


14/14 [==============================] - 1s 104ms/step - loss: 1.5798 - accuracy: 0.3036
Test Accuracy: 0.30


# Only extra dataset no original

In [78]:
# Split into train test and val set
train_texts, tmp_texts, train_labels, tmp_labels = train_test_split(
    df_extra['text'], df_extra['label_bias_0-2'], test_size=0.2, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    tmp_texts,tmp_labels, test_size=0.5, random_state=42
)

# Convert to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_texts.values, train_labels.values))
test_dataset = tf.data.Dataset.from_tensor_slices((test_texts.values, test_labels.values))
val_dataset = tf.data.Dataset.from_tensor_slices((val_texts.values, val_labels.values))


# Shuffle and batch the datasets
batch_size = 8 #10-20 for small dats size
train_dataset = train_dataset.shuffle(len(train_texts)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [79]:
batch_size = 20
classifier_model = build_classifier_model()
# Compile the model
classifier_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

# Train the model with EarlyStopping
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stopping]
)


Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/20
346/346 [==============================] - 122s 331ms/step - loss: 0.8585 - accuracy: 0.6108 - val_loss: 0.7159 - val_accuracy: 0.6575
Epoch 2/20
346/346 [==============================] - 114s 329ms/step - loss: 0.6429 - accuracy: 0.7143 - val_loss: 0.6717 - val_accuracy: 0.6633
Epoch 3/20
346/346 [==============================] - 114s 328ms/step - loss: 0.5500 - accuracy: 0.7660 - val_loss: 0.6568 - val_accuracy: 0.7124
Epoch 4/20
346/346 [==============================] - 113s 328ms/step - loss: 0.4419 - accuracy: 0.8217 - val_loss: 0.6755 - val_accuracy: 0.7052
Epoch 5/20
346/346 [==============================] - 114s 329ms/step - loss: 0.3647 - accuracy: 0.8510 - val_loss: 0.6914 - val_accuracy: 0.7052
Epoch 6/20
346/346 [==============================] - 114s 328ms/step - loss: 0.2529 - accuracy: 0.9096 - val_loss: 0.7790 - val_accuracy: 0.7095
Epoch 7/20
346/346 [============

In [80]:
# Evaluate the model on the test dataset
loss, accuracy = classifier_model.evaluate(test_dataset)

# Print the test accuracy
print(f"Test Accuracy: {accuracy:.2f}")


87/87 [==============================] - 7s 85ms/step - loss: 0.6568 - accuracy: 0.7124
Test Accuracy: 0.71
